## Create a Token on the Algorand Blockchain
#### 04.3 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-11-28

* Create an ASA token
* Transfer an ASA token
* Information about token holdings

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

## Create an ASA = Algorand Standard Asset

This is our first token. The account MyAlgo will create an asset called `WSC Coin`. 

During creation, we need to assign the following roles, which we will use later ...

* **Manager:** can change the thre follwoing roles
* **Reserve:** where not yet distributed assets reside
* **Freeze:** can freeze assets (e.g. to wait for KYC)
* **Clawback:** can undo transactions *if users have opted in*

See https://developer.algorand.org/docs/features/asa/ 

### Step 1: Prepare
* Remember that tokens are divisible
* We choose 2 decimals
    * Our token can be divided into units as small as $\frac{1}{10^2} = \frac{1}{100} = 0.01$
* To create 1000 tokens, we must create $1000 \cdot 10^2$ *small units*

In [4]:
sp = algod_client.suggested_params()
token_supply = 1000                              # Token supply (big units)
token_decimals =  2                              # Digits after the comma
token_total = token_supply * 10**token_decimals  # Specify number of SMALLER unit ("cents")
print(token_total)

token_name  = "Peters WSC coin"                  # <----- YOUR NAME HERE
token_url   = "www.usi.ch/wsc"                   # <----- CHANGE if you want to
token_unit  = "WSC"                              # Abbreviation, e.g. shown in Algorand Wallet app  

100000


### Step 2: Asset creation transaction
New type of transaction, the `AssetConfigTxn`

In [5]:
txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                      
    total=token_total,                         # Total supply in SMALL unit
    decimals=token_decimals,                   # Decimals
    default_frozen=False,                      # Are tokens frozen by default?
    unit_name=token_unit,                      # Abbreviation     
    asset_name=token_name,                     # Name
    url=token_url,                             # URL
    manager=Alice['public'],                   # Special roles (later more)
    reserve=Alice['public'],                   # Special roles
    freeze=Bob['public'],                      # Special roles
    clawback=Bob['public']                     # Special roles
)
print(txn)

{'sender': 'VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4', 'fee': 1000, 'first_valid_round': 26068669, 'last_valid_round': 26069669, 'note': None, 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'acfg', 'rekey_to': None, 'index': 0, 'total': 100000, 'default_frozen': False, 'unit_name': 'WSC', 'asset_name': 'Peters WSC coin', 'manager': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'reserve': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'freeze': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'clawback': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'url': 'www.usi.ch/wsc', 'metadata_hash': None, 'decimals': 2}


### Step 3: Sign and send

In [6]:
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

Y2FQCFF7O7M3W4VX45TPUKDXYNFQXZG5MQJFUGZ35IKWX2UB3CUQ


### Step 4: Wait for confirmation

In [7]:
# Wait for the transaction to be confirmed
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  26068671.
Waiting for round 26068671 to finish.
Transaction Y2FQCFF7O7M3W4VX45TPUKDXYNFQXZG5MQJFUGZ35IKWX2UB3CUQ confirmed in round 26068672.


### Step 5a: Asset index and information
The asset's index is automatically created when the transaction is processed

In [8]:
print(txinfo)

{'asset-index': 147366699, 'confirmed-round': 26068672, 'pool-error': '', 'txn': {'sig': 'UgD7ZpO6eKnDVtnwIW+LcUscE5g8Q4yT2/WrA2CynrCfakd3LN5PtkFmR9S2D9I951825vwT81Eh/YPR3760BQ==', 'txn': {'apar': {'an': 'Peters WSC coin', 'au': 'www.usi.ch/wsc', 'c': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'dc': 2, 'f': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 'm': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 'r': 'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU', 't': 100000, 'un': 'WSC'}, 'fee': 1000, 'fv': 26068669, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 26069669, 'snd': 'VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4', 'type': 'acfg'}}}


In [10]:
# Get the asset ID and open in Algoexplorer
WSC_id = txinfo['asset-index']
print(WSC_id)
print('https://testnet.algoexplorer.io/asset/{}'.format(WSC_id))

147366699
https://testnet.algoexplorer.io/asset/147366699


#### Step 5b: Asset holdings on the wallet app
**CHECK** your wallet app, where the holdings of WSC will already have shown up.

In [ ]:
# We can also check the assets that we have created
account_info = algod_client.account_info(MyAlgo['public'])
json_str = json.dumps(account_info['created-assets'])
json.loads(json_str)

## Transfer coins
In this section we will transfer coins ...
* Manually to Alice
* Using Python to Bob

### Manual transfer to Alice
* Alice has to opt-in
  * Tap `+ Add new asset`, then `All` (not "Only verified")
  * Search for name or asset ID
* Now you can send 10 WSC via QR code or copy-paste
* To send *your* WSC to your neigbhor, she has to opt into your WSC coin!

### Exercise: Python transfer to Bob 
* Remember that transfers of tokens (ASA) are performed with `AssetTransferTxn()` and not with `PaymentTxn()`
* Also, do not forget the ...

In [ ]:
# Your Python code goes here ...




In [ ]:
# More Python code may go here ...




#### A quick check of the (new) holdings of Bob

In [ ]:
account_info = algod_client.account_info(Alice['public'])
account_info

## Appendix: Get (more/better) information
To reduce the confusion, especially when transferring ASA from one account to another, we create an overview of an account's holdings

### Account info
* ALGOs are always treated separately from ASA

In [46]:
print(account_info['amount'])         # micro Algos
print(account_info['amount']/1E6)     # Algos

57824106
57.824106


In [ ]:
account_info['assets']                # All ASA

### Asset Info

In [ ]:
asset_info = algod_client.asset_info(WSC_id)
asset_info

In [49]:
# Useful fields
print(asset_info['params']['decimals'])
print(asset_info['params']['unit-name'])

2
WSC


### Putting it together
* Crate a Python list with all relevant holdings of an account

In [ ]:
info = []
# Algo part
info.append( {'amount':  account_info['amount']/1E6, 'unit' :   'ALGO', 'asset-id': 0, 'name': 'Algorand' } )
# ASA part
assets = account_info['assets']
for asset in assets:
    asset_id     = asset['asset-id']
    asset_info   = algod_client.asset_info(asset_id)                         # Get all info
    asset_amount = asset['amount']/10**asset_info['params']['decimals']      # convert to BIG units
    asset_unit   = asset_info['params']['unit-name']
    info.append( {'amount':  asset_amount,
                  'unit' :   asset_unit,
                  'asset-id':asset_id,
                  'name': asset_info['params']['name']
                  } )

info

### Packaging and adding a data.frame
* `asset_holdings()` returns a list
* `asset_holdings_df()` returns a pandas data frame

The functions are also part of `algo_util.py`.

In [59]:
def asset_holdings(algod_client, public):
    # client = algosdk client
    # public = public address to be analyzed

    import pandas as pd
    from algosdk.v2client import algod
    account_info = algod_client.account_info(public)

    info = []
    # Algo part
    info.append( {'amount':  account_info['amount']/1E6, 
                  'unit' :   'ALGO', 
                  'asset-id': 0, 
                  'name': 'Algorand'
                  } )

    # ASA part
    assets = account_info['assets']
    for asset in assets:
        asset_id     = asset['asset-id']
        asset_info   = algod_client.asset_info(asset_id)                         # Get all info
        asset_amount = asset['amount']/10**asset_info['params']['decimals']      # convert to BIG units
        asset_unit   = asset_info['params']['unit-name']
        info.append( {'amount':  asset_amount,
                      'unit' :   asset_unit,
                      'asset-id':asset_id,
                      'name': asset_info['params']['name']
                      } )
    return(info)

In [ ]:
asset_holdings(algod_client,Alice['public'])

In [61]:
def asset_holdings_df(algod_client, public):
    import pandas as pd
    from algosdk.v2client import algod
    info = asset_holdings(algod_client, public)
    df = pd.DataFrame(info)
    return(df)

In [ ]:
asset_holdings_df(algod_client,Alice['public'])